In [1]:
import os
import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [82]:
#smile file generation for actives
actives = open('/Users/dshrestha/Desktop/Pharos/ligands_obtained/Sodium__and_chloride_dependent_glycine_transporter_2.txt','r').readlines()
active_smiles = open('/Users/dshrestha/Desktop/Pharos/active_glycine_smiles.smi','w')

for lines in actives:
    line = lines.split('\t')[0]
    active_smiles.write(line+'\n')

active_smiles.close()


In [83]:
#smile file generation for decoys
decoy_path ='/Users/dshrestha/Desktop/Pharos/Decoys_set_1/Sodium__and_chloride_dependent_glycine_transporter_2/decoys/'
files = os.listdir(decoy_path)
decoy_smiles = open('/Users/dshrestha/Desktop/Pharos/decoy_glycine_smiles.smi','w')
for file in files:
    if file != '.DS_Store':
        decoys = open(decoy_path+file,'r').readlines() 
        for lines in decoys:
            if not lines.startswith('ligand'):
                line = lines.split('\t')[0]
                decoy_smiles.write(line+'\n')
            else:
                decoy_smiles.write(lines.split('\t')[1]+'\n')
decoy_smiles.close()

In [84]:
#converting smiles to sdf
def smiles2sdf(input_file, output_file):
    os.system('python /Users/dshrestha/Downloads/mayachemtools/bin/RDKitConvertFileFormat.py -i /Users/dshrestha/Desktop/Pharos/'+input_file+' --ov -o /Users/dshrestha/Desktop/Pharos/'+output_file)

smiles2sdf('active_glycine_smiles.smi', 'glycine_actives.sdf')
smiles2sdf('decoy_glycine_smiles.smi', 'glycine_decoys.sdf')

In [2]:
#generating features for actives
#command='perl  /Users/dshrestha/Downloads/mayachemtools/bin/TopologicalPharmacophoreAtomTripletsFingerprints.pl -r /Users/dshrestha/Desktop/Pharos/glycineActivesTPATF --AtomTripletsSetSizeToUse FixedSize -v ValuesString -o /Users/dshrestha/Desktop/Pharos/glycine_actives.sdf'
#os.system(command)

active = open('/Users/dshrestha/Desktop/Pharos/fingerprints/glycineActivesTPATF.csv','r').readlines()
frame1=[]
for lines in active:
    #print(lines)
    if 'Cmpd' in lines:
        #i = re.findall(r'\d+', lines)
        #val=(data.iloc[(int(i[0])-1), 3])
        line = lines.split(';')[5].rstrip('"\n').split(' ')
        #print(len(line))
        df = pd.DataFrame(np.array(line).reshape(1,len(line)))
        df.astype(int)
        frame1.append(df)
active_val = [1]*len(frame1)


In [86]:
#fingerprint generation for decoys
command2='perl  /Users/dshrestha/Downloads/mayachemtools/bin/TopologicalPharmacophoreAtomTripletsFingerprints.pl -r /Users/dshrestha/Desktop/Pharos/glycineDecoysTPATF --AtomTripletsSetSizeToUse FixedSize -v ValuesString -o /Users/dshrestha/Desktop/Pharos/glycine_decoys.sdf'
os.system(command2)


0

In [3]:
#fingerprint generation for decoys
decoy = open('/Users/dshrestha/Desktop/Pharos/fingerprints/glycineDecoysTPATF.csv','r').readlines()
frame2 =[]
for lines in decoy:
    #print(lines)
    if 'Cmpd' in lines:
        #i = re.findall(r'\d+', lines)
        #val=(data.iloc[(int(i[0])-1), 3])
        line = lines.split(';')[5].rstrip('"\n').split(' ')
        #print(len(line))
        df = pd.DataFrame(np.array(line).reshape(1,len(line)))
        df.astype(int)
        frame2.append(df)
decoy_val = [0]*len(frame2)
#frames = frame1 + frame2
#x = pd.concat(frame1, frame2)
#values = active_val+decoy_val
#y = pd.DataFrame(values)


In [4]:
#combining actives and decoys for data splitting
frames = frame1+frame2
len(frames)
x = pd.concat(frames)
#x.to_csv('/Users/dshrestha/Desktop/Pharos/activefingerprints_glycine.csv', sep=',')
values = active_val+decoy_val
print(len(values))
y = pd.DataFrame(values)
print(len(x), len(y))
len(active_val)

4444
(4444, 4444)


58

In [190]:
#splitting the dataset into train and split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20)
list(y_test.values.flatten()).count(1)

15

In [185]:
dew =list(y_train.values.flatten())
dew.count(1)

48

In [191]:
#support vector classification

#t = (np.array(y_train).ravel())

classifier = SVC(C=100.0, kernel ='linear', degree=5, gamma=0.3, coef0=0.0, shrinking=True, 
                 probability=True, tol=0.003, cache_size=200, class_weight=None, verbose=False, 
                 max_iter=-1, decision_function_shape='ovo', random_state=None)
#classifier = SVC(kernel ='linear', probability=True)
classifier.fit(x_train, y_train)
y_predicted = classifier.predict(x_test)
train_pred = classifier.predict(x_train)

train_score= roc_auc_score(y_train, train_pred)
test_score = roc_auc_score(y_test, y_predicted)

train_score2= roc_auc_score(y_train, pd.DataFrame(train_pred))
test_score2 = roc_auc_score(y_test, (y_predicted))

print('test:', test_score)
print('train:', train_score)

#print('test2:', test_score)
#print('train2:', train_score)

print(confusion_matrix(y_test, y_predicted))
print('\n')
#print(t)
y_predicted = map(int, y_predicted)
tn, fp, fn, tp = confusion_matrix(y_test, y_predicted).ravel()
sensitivity = float(tp)/(tp+fn)
specificity = float(tn)/(tn+fp)
PPV = float(tp)/(tp+fp)
print('sensitivity: ', sensitivity)
print('specificity: ', specificity)
print('PPV: ', PPV)

('test:', 0.73047292143401987)
('train:', 0.83692456428457906)
[[869   5]
 [  8   7]]


('sensitivity: ', 0.4666666666666667)
('specificity: ', 0.994279176201373)
('PPV: ', 0.5833333333333334)


In [ ]:
from sklearn.model_selection import GridSearchCV

In [232]:
#Random Forest Classification

RandomClassifier = RandomForestClassifier(n_estimators = 200, criterion = 'entropy')
RandomClassifier.fit(x_train, y_train)

y_predicted = RandomClassifier.predict(x_test)
train_pred = RandomClassifier.predict(x_train)

train_score= roc_auc_score(y_train, pd.DataFrame(train_pred))
test_score = roc_auc_score(y_test, (y_predicted))

print('test:', test_score)
print('train:', train_score)

print(confusion_matrix(y_test, y_predicted))
print('\n')
#print(t)
y_predicted = map(int, y_predicted)
tn, fp, fn, tp = confusion_matrix(y_test, y_predicted).ravel()
sensitivity = float(tp)/(tp+fn)
specificity = float(tn)/(tn+fp)
PPV = float(tp)/(tp+fp)
print('sensitivity: ', sensitivity)
print('specificity: ', specificity)
print('PPV: ', PPV)

/anaconda3/envs/deepchem/lib/python2.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


('test:', 0.73218916857360794)
('train:', 0.84841010224082214)
[[872   2]
 [  8   7]]


('sensitivity: ', 0.4666666666666667)
('specificity: ', 0.9977116704805492)
('PPV: ', 0.7777777777777778)


In [253]:
from sklearn.linear_model import LogisticRegression
Lmodel = LogisticRegression(solver='liblinear', class_weight='balanced', multi_class='ovr',dual=True, C=100, max_iter=100)
Lmodel.fit(x_train, y_train)

y_predicted = Lmodel.predict(x_test)
train_pred = Lmodel.predict(x_train)

train_score = roc_auc_score(y_train, train_pred)
test_score = roc_auc_score(y_test, y_predicted)

print(train_score)
print(test_score)
print(confusion_matrix(y_test, y_predicted))
print('\n')
y_predicted = map(int, y_predicted)
tn, fp, fn, tp = confusion_matrix(y_test, y_predicted).ravel()
sensitivity = float(tp)/(tp+fn)
specificity = float(tn)/(tn+fp)
PPV = float(tp)/(tp+fp)
print('sensitivity: ', sensitivity)
print('specificity: ', specificity)
print('PPV: ', PPV)

print(confusion_matrix(y_train, train_pred))

0.893782777984
0.897139588101
[[869   5]
 [  3  12]]


('sensitivity: ', 0.8)
('specificity: ', 0.994279176201373)
('PPV: ', 0.7058823529411765)
[[3501   11]
 [   9   34]]


In [259]:
from sklearn.ensemble import ExtraTreesClassifier
extra = ExtraTreesClassifier(n_estimators=100, max_features= None, criterion='entropy', bootstrap=True, oob_score=True, 
                             warm_start = True, class_weight ='balanced')
extra.fit(x_train, y_train)

y_predicted = extra.predict(x_test)
train_pred = extra.predict(x_train)

train_score = roc_auc_score(y_train, train_pred)
test_score = roc_auc_score(y_test, y_predicted)

print(train_score)
print(test_score)
print(confusion_matrix(y_test, y_predicted))
print('\n')
y_predicted = map(int, y_predicted)
tn, fp, fn, tp = confusion_matrix(y_test, y_predicted).ravel()
sensitivity = float(tp)/(tp+fn)
specificity = float(tn)/(tn+fp)
PPV = float(tp)/(tp+fp)
print('sensitivity: ', sensitivity)
print('specificity: ', specificity)
print('PPV: ', PPV)

print(confusion_matrix(y_train, train_pred))

/anaconda3/envs/deepchem/lib/python2.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


0.99729498861
0.897711670481
[[870   4]
 [  3  12]]


('sensitivity: ', 0.8)
('specificity: ', 0.9954233409610984)
('PPV: ', 0.75)
[[3493   19]
 [   0   43]]


In [257]:
from sklearn.ensemble import VotingClassifier
estimators = []

estimators.append(('logistic', Lmodel))
#estimators.append(('svm', classifier))
estimators.append(('RF', extra))
ensemble = VotingClassifier(estimators, voting='soft')
ensemble.fit(x_train, y_train)

y_predicted = ensemble.predict(x_test)
train_pred = ensemble.predict(x_train)

train_score = roc_auc_score(y_train, train_pred)
test_score = roc_auc_score(y_test, y_predicted)

print(train_score)
print(test_score)
print(confusion_matrix(y_test, y_predicted))
print('\n')
y_predicted = map(int, y_predicted)
tn, fp, fn, tp = confusion_matrix(y_test, y_predicted).ravel()
sensitivity = float(tp)/(tp+fn)
specificity = float(tn)/(tn+fp)
PPV = float(tp)/(tp+fp)
print('sensitivity: ', sensitivity)
print('specificity: ', specificity)
print('PPV: ', PPV)

0.767441860465
0.766666666667
[[874   0]
 [  7   8]]


('sensitivity: ', 0.5333333333333333)
('specificity: ', 1.0)
('PPV: ', 1.0)


In [258]:
print(confusion_matrix(y_train, train_pred))
print('\n')
train_pred = map(int, train_pred)
tn1, fp1, fn1, tp1 = confusion_matrix(y_train, train_pred).ravel()
sensitivity = float(tp1)/(tp1+fn1)
specificity = float(tn1)/(tn1+fp1)
PPV = float(tp1)/(tp1+fp1)
print('sensitivity: ', sensitivity)
print('specificity: ', specificity)
print('PPV: ', PPV)

[[3512    0]
 [  20   23]]


('sensitivity: ', 0.5348837209302325)
('specificity: ', 1.0)
('PPV: ', 1.0)


In [260]:
#saving the model
pkl_filename = "/Users/dshrestha/Desktop/Pharos/Models/extra_tree_glycine_transporter2.pkl"
pickle.dump(extra, open(pkl_filename, 'wb'))
#with open(pkl_filename, 'wb') as file:
#    pickle.dump(classifier, file)